## Import packages

In [1]:
cd /Users/zoltan/Dropbox/Python

/Users/zoltan/Dropbox/Python


In [2]:
pwd

u'/Users/zoltan/Dropbox/Python'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import cline_analysis as ca
import pandas as pd
import seaborn as sns
import datetime
import os
from scipy.signal import medfilt
import functools
from scipy.optimize import bisect
from scipy import stats

sns.set_style("whitegrid")
sns.set_style("ticks")
%matplotlib qt
%config InlineBackend.figure_format = 'svg'
plt.matplotlib.rcParams['svg.fonttype'] = 'svgfont' # fonts will be recognized by Adobe Illustrator

In [15]:
reload(ca)

<module 'cline_analysis' from 'cline_analysis.py'>

## Load data

In [16]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Bolivia/Mamore/csv_files_new/'
fnames,clxs,clys,rbxs,lbxs,rbys,lbys,curvatures,ages,widths,dates = ca.load_data(dirname)

In [17]:
fnames

['Mamore_19840811.csv',
 'Mamore_19861105.csv',
 'Mamore_19890708.csv',
 'Mamore_19900524.csv']

In [6]:
dates

[datetime.datetime(1984, 8, 11, 0, 0),
 datetime.datetime(1986, 11, 5, 0, 0),
 datetime.datetime(1989, 7, 8, 0, 0),
 datetime.datetime(1990, 5, 24, 0, 0)]

## Get migration rate

In [7]:
ts1 = 1 # first timestep
ts2 = 2 # second timestep

d = dates[ts2]-dates[ts1]
years = d.days/365.0

x = np.array(clxs[ts1])
y = np.array(clys[ts1])

xn = np.array(clxs[ts2])
yn = np.array(clys[ts2])

migr_rate, migr_sign, p, q = ca.get_migr_rate(x,y,xn,yn,years,0)

In [10]:
# qc
plt.figure()
plt.plot(x,y)
plt.plot(xn,yn)
plt.axis('equal');

In [8]:
curv,s = ca.compute_curvature(x,y)
plt.figure()
plt.plot(s,curv)
curv = medfilt(savgol_filter(curv,71,3),kernel_size=5) # smoothing
plt.plot(s,curv);

In [9]:
plt.figure()
plt.plot(s,migr_rate)
migr_rate = medfilt(savgol_filter(migr_rate,41,3),kernel_size=5) # smoothing
plt.plot(s,migr_rate);

In [14]:
# set intervals affected by cu=toffs to NaN - specific to Mamore river 
migr_rate[907:1519] = np.NaN
migr_rate[12053:12424] = np.NaN
migr_rate[16809:17128] = np.NaN

In [45]:
# qc
plt.figure()
plt.plot(migr_rate)

In [12]:
from shapely.geometry import LineString
import geopandas as gpd

def write_shapefile(x,y,epsg,dirname,filename):
    coords=[]
    for i in range(len(x)):
        coords.append((x[i],y[i]))
    ls = LineString(coords)
    ls = gpd.GeoSeries(ls)
    ls.crs = {'init' :'epsg:'+str(epsg)}
    ls.to_file(dirname+filename+'.shp')

In [13]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Curvature_paper_II/GIS_data/Mamore/'
filename = str(dates[ts1].year)+str(dates[ts1].month)+str(dates[ts1].day)+'_cl'
write_shapefile(x,y,epsg=32620,dirname=dirname,filename=filename)

In [14]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Curvature_paper_II/GIS_data/Mamore/'
filename = str(dates[ts2].year)+str(dates[ts2].month)+str(dates[ts2].day)+'_cl'
write_shapefile(xn,yn,epsg=32620,dirname=dirname,filename=filename)

In [35]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Curvature_paper_II/GIS_data/Mamore/'
epsg = 32620

year = str(dates[ts1].year)
day = str(dates[ts1].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts1].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(x,y,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts1],lbys[ts1],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts1],rbys[ts1],epsg=epsg,dirname=dirname,filename=filename)

year = str(dates[ts2].year)
day = str(dates[ts2].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts2].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(xn,yn,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts2],lbys[ts2],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts2],rbys[ts2],epsg=epsg,dirname=dirname,filename=filename)

## Read 'valid' inflection points and corresponding points of zero migration from CSV file

In [16]:
df = pd.read_csv('MamoreLT05_L1TP_232071_19861105_20170216_01_T1_inflection_and_zero_migration_indices.csv')
LZC = np.array(df['index of inflection point'])
LZM = np.array(df['index of zero migration'])

In [17]:
# indices of bends affected by low erodibility and cutoffs (these have been picked manually)
erodibility_inds = [69,77,149,153,157,159]
cutoff_inds = [61,100,101,107,116,147,148]

## Plot curvature and migration rate series side-by-side

In [18]:
# plot curvature and migration rate along the channel

W = np.nanmean(widths[1])

fig, ax1 = plt.subplots(figsize=(25,4))
plt.tight_layout()

curv_scale = 0.6
migr_scale = 60
y1 = curv_scale
y2 = -3*curv_scale
y3 = 3*migr_scale
y4 = -migr_scale

y5 = -2*curv_scale
y6 = 2*migr_scale

for i in range(0,len(LZC)-1,2):
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,0.85,0.85],edgecolor='k',zorder=0)

deltas = 25.0
ax1.fill_between(s, 0, curv*W)
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_rate, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y2,y1)
ax2.set_ylim(y4,y3)
ax1.set_xlim(0,s[-1])

for i in erodibility_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[1.0,0.85,0.85],edgecolor='k',zorder=0) 
    
for i in cutoff_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,1.0,0.85],edgecolor='k',zorder=0) 
    
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y1,y1,0,y5,y2,y2,y5,0]
        ax1.fill(xcoords,ycoords,color='w') 
        
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y3,y3,y6,0,y4,y4,0,y6]
        ax2.fill(xcoords,ycoords,color='w') 

for i in range(0,len(LZC)-1,2):
    ax1.text(s[LZC[i]],0.5,str(i),fontsize=12)

## Estimate lag between curvature and migration rate

In [20]:
# plot widths and boundary between the two segments
plt.figure()
plt.plot(s,widths[1])
plt.plot([s[10516],s[10516]],[0,800],'r')

In [22]:
# first segment
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(widths[1][:10516])

226.15607678950275

In [23]:
# second segment
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(widths[1][10516:])

314.2718470671267

In [24]:
np.mean(25.0*(LZM[:101]-LZC[:101]))

647.2772277227723

In [25]:
np.mean(25.0*(LZM[101:]-LZC[101:]))

911.69354838709683

# First segment (Mamore A)

## Estimate friction factor Cf

In [27]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 1519
i2 = 10516
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [31]:
plt.figure()
plt.plot(migr_rate_t)

In [28]:
# this might take a while to run
kl = 60.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[1][i1:i2])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.00760703125


In [29]:
Cf_opt = 0.00760703125

## Estimate migration rate constant kl

In [31]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
for i in np.arange(30,80):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(30,80),errors);

30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [32]:
kl_opt = 60.0 # the error is at minimum for kl = 60.0

In [33]:
647/25.0

25.88

In [44]:
plt.figure()
plt.plot(W*kl_opt*curv_t)
plt.plot(migr_rate_t)

## Plot actual migration rate against nominal migration rate

In [36]:
# kernel density and scatterplot of actual vs. nominal migration rate
w = np.mean(widths[1][i1:i2]) 

curv_nodim = W*curv_t*kl_opt
lag = 26
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 50.0
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--') 
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [37]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.778991262865
0.606827387619
0.0


In [38]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

8971

In [39]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [40]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r')

In [41]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.771376417234
0.595021577065
0.0


In [42]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

38.974032537357473

In [43]:
# plot actual vs. predicted migration rate
max_m = 50.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [44]:
# plot actual vs. predicted migration rate
max_m = 70.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    if (i>=8) and (i<101):
        plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::10],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::10],c='r',s=15)
for i in cutoff_inds:
    if (i>=8) and (i<101):
        plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::10],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::10],c='g',s=15)

# Second segment (Mamore B)

## Estimate friction factor Cf

In [50]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 10516
i2 = len(x)-1
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [51]:
migr_rate_t[12053-i1:12424-i1] = 0
migr_rate_t[16809-i1:17128-i1] = 0
plt.figure()
plt.plot(migr_rate_t)

In [52]:
# this might take a while to run
kl = 100.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[1][i1:i2])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
curv_t[12053-i1:12424-i1] = 0
curv_t[16809-i1:17128-i1] = 0

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.00448828125


In [53]:
Cf_opt = 0.00448828125

In [58]:
w

227.33329569767974

## Estimate migration rate constant kl

In [54]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
for i in np.arange(120,160):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(120,160),errors);

120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


In [59]:
kl_opt = 128.0 # the error is at minimum for kl = 128.0

In [55]:
912/25.0 # lag

36.48

In [60]:
plt.figure()
plt.plot(W*kl_opt*curv_t)
plt.plot(migr_rate_t)

## Plot actual migration rate against nominal migration rate

In [68]:
migr_rate_t[12053-i1:12424-i1] = np.NaN
migr_rate_t[16809-i1:17128-i1] = np.NaN

# kernel density and scatterplot of actual vs. nominal migration rate
w = np.nanmean(widths[1][i1:i2]) 

curv_nodim = W*curv_t*kl_opt
lag = 36
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 100.0
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--') 
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [69]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.641711040854
0.411793059954
0.0


In [70]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

7341

In [71]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [72]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r')

In [73]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.625278959609
0.39097377733
0.0


In [74]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

82.04226937039796

In [75]:
# plot actual vs. predicted migration rate
max_m = 100.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [80]:
# plot actual vs. predicted migration rate
max_m = 120.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    if i>=101:
        plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::5],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::5],c='r',s=15)
for i in cutoff_inds:
    if i>=101:
        plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::5],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::5],c='g',s=15)